In [ ]:
#| default_exp core

# fhswiftui.core

> API for helpers, layout components, UI components, and style modifiers.

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
import fasthtml.components as fh
from fasthtml.jupyter import *

In [ ]:
#| export
#| hide
def _def_colors():
    base = ["primary","secondary","accent","muted","card","popover"]
    return base+[f"{o}-foreground" for o in base]+["background","foreground","destructive","ring","input","border"]
    

## Basics

In [ ]:
#| export
def IncludeColors(
    colors:list=None,  # Additional Tailwind color names to include (e.g., ['red-500', 'blue-300'])
    append:bool=True,  # Append to default theme colors; set `False` to replace all defaults
): # Hidden div with color classes to ensure they're included in Tailwind's build
    "Include additional colors that will be available via Tailwind utility classes on the page"
    from itertools import product
    if not colors: colors = []
    pr = [f"border-{o}" for o in "lrtb"] + ["bg", "text", "border"]
    return Div(cls=f"hidden {' '.join(f'{p}-{c}' for p,c in product(pr, _def_colors() + colors if append else colors))}")

In [ ]:
#| export
# enable basecoat and tailwind; add useful default colors
bc_link = Link(rel='stylesheet', href='https://cdn.jsdelivr.net/npm/basecoat-css@latest/dist/basecoat.cdn.min.css')
tw_scr = Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4')

fh_swiftui_hdrs = (bc_link,tw_scr,IncludeColors())


In [ ]:

#| hide
fh_swiftui_hdrs

(link((),{'rel': 'stylesheet', 'href': 'https://cdn.jsdelivr.net/npm/basecoat-css@latest/dist/basecoat.cdn.min.css'}),
 script(('',),{'src': 'https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'}),
 div((),{'class': 'hidden border-l-primary border-l-secondary border-l-accent border-l-muted border-l-card border-l-popover border-l-primary-foreground border-l-secondary-foreground border-l-accent-foreground border-l-muted-foreground border-l-card-foreground border-l-popover-foreground border-l-background border-l-foreground border-l-destructive border-l-ring border-l-input border-l-border border-r-primary border-r-secondary border-r-accent border-r-muted border-r-card border-r-popover border-r-primary-foreground border-r-secondary-foreground border-r-accent-foreground border-r-muted-foreground border-r-card-foreground border-r-popover-foreground border-r-background border-r-foreground border-r-destructive border-r-ring border-r-input border-r-border border-t-primary border-t-secondary bord

In [ ]:
#| export
def mk_previewer(
    app=None,  # FastHTML app instance; defaults to app with fh_swiftui_hdrs
    hdrs=None, # Custom headers (unused, kept for API compatibility)
    cls:str='max-w-lg', # Tailwind max-width class (e.g., 'max-w-sm', 'max-w-xl', 'max-w-2xl')
): # Returns a previewer function `p(*c, cls='', **kw)` for rendering components in an iframe
    "Create a previewer function for rendering FastHTML components in iframes"
    xcls = cls
    if not app: app=FastHTML(hdrs=fh_swiftui_hdrs)
    def p(*c, cls='', **kw):
        return HTMX(Div(cls=f'{xcls} {cls}')(*c), app=app, host=None, port=None, **kw)
    return p

Note: mk_previewer is derived from: [ghdaisy](https://answerdotai.github.io/fhdaisy/core.html#mk_previewer) 

In [ ]:
p = mk_previewer()

In [ ]:
c = Button("hello", cls="btn")
p(c)

In [ ]:
c

```html
<button class="btn">hello</button>
```

## Style modifiers

In [ ]:
#| export
@patch
def append_classes(
    self:FT, # FastTag element to modify
    *c       # CSS class names to append
): # Modified FastTag element with classes appended
    "Append css classes to the FastTag element"
    self.attrs["class"] = " ".join(f"{self.attrs.get('class','')} {' '.join(c)}".split())
    return self

### Padding - space inside the component

In [ ]:
#| export
@patch
def padding(
    self:FT, # FastTag element to modify
    **kw,    # Padding values: 'all', 'top', 'bottom', 'left', 'right'. Values use Tailwind spacing (0-96, px, or '[<custom>]')
): # Modified FastTag with padding classes appended
    "Add padding around the content within the element's bounds. Defaults to p-4 if no arguments provided."
    d = {"top":'t', "bottom":'b', "left":'l', "right":'r'}
    c = [f"p-{kw['all']}"] if "all" in kw else [f"p{d[k]}-{kw[k]}" for k in set(kw) & d.keys()]
    return self.append_classes(*c if c else ["p-4"])

#### Tests

In [ ]:
# Test with different padding options
Div("Default", cls="bg-blue-200").padding()

```html
<div class="bg-blue-200 p-4">Default</div>

```

In [ ]:
# Test with different padding options
p(Div("Default", cls="bg-blue-200").padding())

In [ ]:
Div("All sides", cls="bg-blue-200").padding(all=6)

```html
<div class="bg-blue-200 p-6">All sides</div>

```

In [ ]:
p(Div("All sides", cls="bg-blue-200").padding(all=6))

In [ ]:
Div("Top and bottom", cls="bg-blue-200").padding(top=2, bottom=8)

```html
<div class="bg-blue-200 pb-8 pt-2">Top and bottom</div>

```

In [ ]:
p(Div("Top and bottom", cls="bg-blue-200").padding(top=2, bottom=8))

### Margins - space outside the component

In [ ]:
#| export
@patch
def margin(
    self:FT, # FastTag element to modify
    **kw,    # Margin values: 'all', 'top', 'bottom', 'left', 'right'. Values use Tailwind spacing (0-96, auto, px, or '[<custom>]')
): # Modified FastTag with margin classes appended
    "Add margin (space outside) to the element. Defaults to m-4 if no arguments provided."
    d = {"top":'t', "bottom":'b', "left":'l', "right":'r'}
    c = [f"m-{kw['all']}"] if "all" in kw else [f"m{d[k]}-{kw[k]}" for k in set(kw) & d.keys()]
    return self.append_classes(*c if c else ["m-4"])

#### Tests

In [ ]:
# Default margin
p(Div("Default", cls="bg-blue-200").margin())

In [ ]:
# All sides
p(Div("All sides", cls="bg-blue-200").margin(all=6))

In [ ]:
# Top and bottom
p(Div("Top and bottom", cls="bg-blue-200").margin(top=2, bottom=8))

### Borders

In [ ]:
#| export
@patch
def border(
    self:FT,        # FastTag element to modify
    width:int=None, # Border width as integer (0, 2, 4, 8, etc.) producing border-<width> class
    color:str=None, # Border color without 'border-' prefix (e.g., 'primary', 'muted', 'gray-300')
): # Modified FastTag with border classes appended
    "Add a border around the element with optional width and color customization"
    c = ["border"]
    if width: c.append(f"border-{width}")
    if color: c.append(f"border-{color}")
    return self.append_classes(*c)

#### Tests

In [ ]:
p(Div("Test").border())

In [ ]:
p(Div("Test").border(width=2,color="blue-500"))

In [ ]:
p(Div("Test").border(color="red-300"))

In [ ]:
p(Div("Test").border().padding().margin())

### Corner radius

In [ ]:
#| export
@patch
def corner_radius(
    self:FT,       # FastTag element to modify
    size:str=None, # Tailwind size suffix: 'xs', 'sm', 'md', 'lg', 'xl', '2xl', '3xl', '4xl', 'none', 'full', or '[<custom>]'
): # Modified FastTag with rounded corner class appended
    "Round the corners of this element. Defaults to 'rounded' (base radius) if no size provided."
    return self.append_classes(f"rounded-{size}" if size else "rounded")

#### Tests

In [ ]:
# Default rounded
p(Div("Default", cls="bg-blue-200 p-4").corner_radius())

In [ ]:
# Specific sizes
p(Div("Small", cls="bg-green-200 p-4").corner_radius("sm"))

In [ ]:
p(Div("Large", cls="bg-red-200 p-4").corner_radius("lg"))

In [ ]:
p(Div("Full", cls="bg-yellow-200 p-4").corner_radius("full"))

In [ ]:
p(Div("Chaining").padding().border().corner_radius("lg"))

### bg/fg - set background and foreground colors

In [ ]:
#| export
@patch
def bg(
    self:FT,  # FastTag element to modify
    color:str, # Tailwind color (e.g., 'primary', 'muted', 'blue-500', 'red-300', or theme colors like 'background')
): # Modified FastTag with background color class appended
    "Set background color using Tailwind bg-<color> utility class"
    return self.append_classes(f"bg-{color}")

In [ ]:
#| export
@patch
def fg(
    self:FT,   # FastTag element to modify
    color:str, # Tailwind color (e.g., 'primary', 'muted', 'blue-500', 'red-300', or theme colors like 'foreground')
): # Modified FastTag with text color class appended
    "Set foreground (text) color using Tailwind text-<color> utility class"
    return self.append_classes(f"text-{color}")

#### Tests

In [ ]:
# Background color
p(Div("Blue background").padding().bg("blue-500"))

In [ ]:
# Foreground/text color
p(Div("Red text").padding().fg("red-600"))

In [ ]:
# Combined
p(Div("Yellow on purple", cls="p-4").bg("purple-700").fg("yellow-300"))

In [ ]:
# With other modifiers
p(Div("Styled box", cls="p-4").bg("green-200").fg("green-900").corner_radius("lg").border(width=2, color="green-500"))

### shadow - set shadow

In [ ]:
#| export
@patch
def shadow(
    self:FT, # FastTag element to modify
    **kw,    # Shadow options: 'size' (2xs/xs/sm/md/lg/xl/2xl/none), 'color' (CSS color), 'x'/'y'/'blur'/'spread' (int px offsets)
): # Modified FastTag with shadow class appended
    "Add shadow to element. Use 'size' for presets, or 'x'/'y'/'blur'/'spread'/'color' for custom shadows."
    if "color" not in kw: kw["color"] = "rgba(0,0,0,0.1)"
    if len(kw) == 1 and "color" in kw: return self.append_classes("shadow")
    if "size" in kw: return self.append_classes(f"shadow-{kw['size']}")
    def enc(color, x=0, y=1, blur=3, spread=0, **k2): return f"shadow-[{x}px_{y}px_{blur}px_{spread}px_{color}]"
    return self.append_classes(enc(**kw))

#### Tests

In [ ]:
# Default shadow
p(Button("Default Shadow", cls="btn btn-primary").shadow())

In [ ]:
# Custom color shadow
p(Button("Blue Shadow", cls="btn").shadow(c="rgba(0,0,255,0.5)", x=2, y=2, blur=8))

In [ ]:
# Large offset shadow
p(Button("Offset Shadow", cls="btn btn-secondary").shadow(x=4, y=8, blur=12))

In [ ]:
# Red shadow
p(Button("Red Shadow", cls="btn btn-accent").shadow(color="red", blur=10))

### opacity - set opacity

In [ ]:
#| export
@patch
def opacity(
    self:FT,    # FastTag element to modify
    pct:int=100, # Opacity percentage (0, 5, 10, 15, 20, 25, 30, ..., 95, 100, or '[<custom>]' as string)
): # Modified FastTag with opacity class appended
    "Set element opacity using Tailwind opacity-<pct> utility class"
    return self.append_classes(f"opacity-{pct}")

#### Tests

In [ ]:
p(Div("Full opacity (default)", cls="bg-blue-500 p-4 text-white").opacity())

In [ ]:
p(Div("75% opacity", cls="bg-red-500 p-4 text-white").opacity(75))

In [ ]:
p(Div("50% opacity", cls="bg-green-500 p-4 text-white").opacity(50))

In [ ]:
p(Div("25% opacity", cls="bg-purple-500 p-4 text-white").opacity(25))

In [ ]:
p(Div(
    Div("100", cls="bg-blue-500 p-4 text-white").opacity(100),
    Div("75", cls="bg-blue-500 p-4 text-white").opacity(75),
    Div("50", cls="bg-blue-500 p-4 text-white").opacity(50),
    Div("25", cls="bg-blue-500 p-4 text-white").opacity(25)
))

### frame - position this within a frame

In [ ]:
#| export
@patch
def frame(
    self:FT,           # FastTag element to wrap in a frame
    w=None,            # Frame width: number (0-96), container size (xs-7xl), 'full', 'screen', 'auto', fraction, or '[<custom>]'
    h=None,            # Frame height: number (0-96), container size (xs-7xl), 'full', 'screen', 'auto', fraction, or '[<custom>]'
    halign:str="center", # Horizontal alignment: 'leading', 'center', 'trailing'
    valign:str="center", # Vertical alignment: 'top', 'center', 'bottom'
): # Div wrapping self with flex alignment and optional size classes
    "Position this element within a flex frame with optional sizing and alignment"
    h_map = {"leading": "justify-start", "center": "justify-center", "trailing": "justify-end"}
    v_map = {"top": "items-start", "center": "items-center", "bottom": "items-end"}
    c = ["flex", h_map[halign], v_map[valign]]
    if w: c.append(f"w-{w}")
    if h: c.append(f"h-{h}")
    return Div(self).append_classes(*c)

#### Tests

In [ ]:
# Basic frame with size
p(Div("Content", cls="bg-blue-200 p-2").frame(w=64, h=32))

In [ ]:
# Different alignments
p(Div("Top Left", cls="bg-red-200 p-2").frame(w=96, h=48, valign="top",halign="leading"))

In [ ]:
p(Div("Center", cls="bg-green-200 p-2").frame(w=96, h=48, halign="center"))

In [ ]:
p(Div("Bottom Right", cls="bg-yellow-200 p-2").frame(w=96, h=48, valign="bottom",halign="trailing"))

In [ ]:
# Just width or height
p(Div("Wide", cls="bg-purple-200 p-2").frame(w="full"))

In [ ]:
p(Div("Tall", cls="bg-pink-200 p-2").frame(h=64))

### font - set the default font for the tag

In [ ]:
#| export
@patch
def font(
    self:FT,       # FastTag element to modify
    size:str=None, # Font size: 'xs', 'sm', 'base', 'lg', 'xl', '2xl', '3xl', '4xl', '5xl', '6xl', '7xl', '8xl', '9xl', or '[<custom>]'
    weight:str=None, # Font weight: 'thin', 'extralight', 'light', 'normal', 'medium', 'semibold', 'bold', 'extrabold', 'black', or '[<custom>]'
): # Modified FastTag with font size and/or weight classes appended
    "Set font size and/or weight using Tailwind text-<size> and font-<weight> utility classes"
    c = []
    if size: c.append(f"text-{size}")
    if weight: c.append(f"font-{weight}")
    return self.append_classes(*c)

#### Tests

In [ ]:
# Different font sizes
p(Div("Small text").font(size="sm"))

In [ ]:
p(Div("Base text").font(size="base"))

In [ ]:
p(Div("Large text").font(size="lg"))

In [ ]:
p(Div("Extra large text").font(size="xl"))

In [ ]:
p(Div("2XL text").font(size="2xl"))

In [ ]:
# Different font weights
p(Div("Light text").font(size="light"))

In [ ]:
p(Div("Normal text").font(weight="normal"))

In [ ]:
p(Div("Bold text").font(weight="bold"))

In [ ]:
# Combined size and weight
p(Div("Large and bold").font(size="2xl",weight="bold"))

In [ ]:
p(Div("Small and light").font(size="sm",weight="light"))